In [ ]:
import dotenv, os, pandas as pd, ollama, csv

percorso_credenziali = '../credenziali/credenziali_cartella.env'
dotenv.load_dotenv(override = True, dotenv_path = percorso_credenziali)

cartella = os.getenv('Download')

sostituisci_colonne = {
    "Prezzo": "Prezzo_unitario",
    "Quantità": "Quantita"
}

ordine_mesi = ["Settembre", "Ottobre"]

file = f'{cartella}\\simulazione_vendite.csv'
vendite = pd.read_csv(file)

file = f'{cartella}\\capstonehp.csv'
catalogo = pd.read_csv(file)

In [ ]:
dataset = pd.merge(vendite, catalogo, on=['Paese', 'Sku'], how='left')

dataset['Prezzo_tot'] = dataset['Prezzo'] * dataset['Quantità']
dataset['Mese'] = dataset['Data'].str[:7]
dataset['Mese'] = (dataset['Mese']
    .str.replace('2025-09', ordine_mesi[0], regex=False)
    .str.replace('2025-10', ordine_mesi[1], regex=False)
)
dataset = dataset.rename(columns=sostituisci_colonne)

In [ ]:
file = f'{cartella}\\capstone_dataset.csv'

dataset.to_csv(file, index=False)

In [ ]:
visualizza = dataset.copy()

visualizza['Mese'] = pd.Categorical(
    visualizza['Mese'],
    categories=ordine_mesi,
    ordered=True
)

visualizza = (
    visualizza.groupby(
        ['Paese','Mese','Categoria','Ambito'],
        observed=True
    ).agg(
        Ricavo=('Prezzo_tot', 'sum'),
        Num_ordini=('ID_Ordine', 'nunique'),
        Prodotti_venduti=('Sku', 'nunique'),
    ).reset_index()
)

visualizza['Ricavo_per_ordine'] = (visualizza['Ricavo'] / visualizza['Num_ordini']).round(2)

visualizza

In [ ]:
inizio_richiesta = 'Sei un assistente che genera executive summary sintetici e leggibili per un business plan basato sui seguenti dati in formato csv di vendita nel paese '

fine_richiesta = '''Analizza le informazioni e crea un executive summary chiaro e strutturato in sezioni, come se fosse destinato a un report aziendale per dirigenti.

Il testo deve essere in italiano, formale ma sintetico, e includere le seguenti parti:

Sintesi generale sull’andamento delle vendite confrontando i mesi di settembre 2025 e ottobre 2025.

Analisi per categoria di prodotto con variazioni percentuali e commenti sulle performance.

Analisi per ambito (Business vs Home).

Indicatori chiave (ricavo medio per ordine, numero ordini, prodotti venduti, variazioni mensili).

Conclusioni e raccomandazioni strategiche per il prossimo periodo basate sulla executive summary appena scritta.

Il risultato deve essere ordinato e leggibile, con punti elenco dove utile e non in formato Markdown.'''

In [ ]:
paesi = visualizza['Paese'].unique()
commenti = []

os.environ["OLLAMA_DEVICE"] = "cpu" # solo per il mio computer

for paese in paesi:
    inizio_richiesta_paese = f'{inizio_richiesta}{paese}.'
    
    visualizza_paese = visualizza.query("Paese == @paese")
    stringa = visualizza_paese.to_csv(index=False, float_format='%.2f')
    
    risposta = ollama.Client().generate(
        model="llama3.2:1b",
        prompt=f"{inizio_richiesta_paese}\n\n{stringa}\n\n{fine_richiesta}",
        options={
            "temperature": 0.2
        }
    ).response
    
    commenti.append({
        'Paese': paese,
        'Valutazione GenAI': risposta
    })

os.system("ollama stop all") # solo per il mio computer

In [ ]:
file = f'{cartella}\\risposte_GenAI.csv'

In [ ]:
with open(file, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=[
        "Paese",
        "Valutazione GenAI"
    ])
    writer.writeheader()
    writer.writerows(commenti)